In [1]:
from scripts.data_generation.well import Well, sample_well
from manywells.simulator import SSDFSimulator
import SPSA

In [2]:
# Check well conditions
well = sample_well()
sim = SSDFSimulator(well_properties=well.wp, boundary_conditions=well.bc)

print(well)

Well(wp=WellProperties(L=1985.7908818477706, D=0.1397, rho_l=911.3310087439986, R_s=485.8316790722516, cp_g=2225, cp_l=2339.216071112378, f_D=0.07960315849191575, h=17.37166268914084, slip=SlipModel(), inflow=Vogel(w_l_max=117.59955527024063, f_g=0.007565255315451339), choke=SimpsonChokeModel(K_c=0.0019504518382759829, cpr=0.544465891827854, chk_profile='concave', _chk_profiles=['linear', 'sigmoid', 'convex', 'concave'])), bc=BoundaryConditions(p_r=198.08617323097684, p_s=24.281341157950543, T_r=347.7237264554331, T_s=277.15, u=1.0, w_lg=0.0), gas=GasProperties(name='gas', R_s=485.8316790722516, cp=2225), oil=LiquidProperties(name='oil', rho=895.1921797390702, cp=2000), water=LiquidProperties(name='water', rho=999.1, cp=4184), fractions=(0.007565255315451339, 0.823445697503501, 0.16898904718104765), has_gas_lift=True)


In [ ]:
x = sim.simulate()
x2_dp = SPSA.create_data_point(well=well, sim=sim, x=x, decision_type='Perturbation')
print(x2_dp["WOIL"].values[0])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

6.878065491210256


CasADi - 2025-04-09 10:16:52 WARNING("S:nlp_jac_g failed: NaN detected for output jac_g_x, at nonzero index 4092 (row 703, col 700).") [.../casadi/core/oracle_function.cpp:377]


In [4]:
x = sim.simulate()
x2_dp = SPSA.create_data_point(well=well, sim=sim, x=x, decision_type='Perturbation')
print(x2_dp["WOIL"].values[0])

6.878065491210256


CasADi - 2025-04-09 10:17:21 WARNING("S:nlp_jac_g failed: NaN detected for output jac_g_x, at nonzero index 4092 (row 703, col 700).") [.../casadi/core/oracle_function.cpp:377]


In [2]:
import pandas as pd

In [ ]:
# Script to delete ghost rows in the data
drops = []
p_counter = 0
start_drop = 0
for i in range(len(df)):
    if df["DCNT"].iloc[i] == "Perturbation":
        p_counter += 1
    elif df["DCNT"].iloc[i] == "Optimizing":
        if p_counter > 3:
            drops.append(list([start_drop, i]))
        p_counter = 0
    if p_counter == 3:
        start_drop = i

for tup in drops:
    df = df.drop(index=range(tup[0], tup[1]))
df = df.reset_index(update=True)

NameError: name 'df' is not defined

In [14]:
import os
import pickle
from itertools import combinations

def files_are_equivalent(f1, f2):
    try:
        with open(f1, 'rb') as a, open(f2, 'rb') as b:
            return pickle.load(a) == pickle.load(b)
    except Exception as e:
        print(f"Error comparing {f1} and {f2}: {e}")
        return False

def remove_pickle_duplicates(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    keep = set()
    deleted = set()

    for i, f1 in enumerate(files):
        if f1 in deleted:
            continue
        keep.add(f1)
        for f2 in files[i+1:]:
            if f2 in deleted or f2 in keep:
                continue
            if files_are_equivalent(f1, f2):
                print(f"Duplicate detected:\n  KEEP: {f1}\n  DELETE: {f2}")
                os.remove(f2)
                deleted.add(f2)

    print(f"\nDeleted {len(deleted)} duplicate files.")
    return deleted

In [16]:
remove_pickle_duplicates("../results/param_set_1/run9/iteration_100/dump")

Error comparing ../results/param_set_1/run9/iteration_100/dump/3dcd8e8b-d853-4cf9-8175-236a55881b52 and ../results/param_set_1/run9/iteration_100/dump/.DS_Store: invalid load key, '\x00'.
Error comparing ../results/param_set_1/run9/iteration_100/dump/3dcd8e8b-d853-4cf9-8175-236a55881b52 and ../results/param_set_1/run9/iteration_100/dump/704ac24a-bfa0-4499-bcc6-85f133b8f3e9: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error comparing ../results/param_set_1/run9/iteration_100/dump/de947dad-c34e-4f83-86c6-1a7a0923f203 and ../results/param_set_1/run9/iteration_100/dump/.DS_Store: invalid load key, '\x00'.
Error comparing ../results/param_set_1/run9/iteration_100/dump/de947dad-c34e-4f83-86c6-1a7a0923f203 and ../results/param_set_1/run9/iteration_100/dump/a8eda29a-2bb6-41d8-9436-b1c8d372bea2: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error comparing ../results/param_set_1/run9/iterat

set()

### Scripts to find datasets

In [3]:
from SPSA import configure_wells, create_data_point, SPSAOptimization
from manywells.simulator import SSDFSimulator, SimError
import pandas as pd

In [21]:
dataset_filename = 'data/testing/SPSA-sol_configs/full.csv'
df = pd.read_csv(dataset_filename)

num = 40
df = df[(df['wp.inflow.w_l_max'] < 60) & (df['has_gas_lift'] == True)].sample(n=num)
wells = configure_wells(df)

differences = pd.DataFrame(columns=['well_id', 'oil_before', 'oil_after', 'difference'])

for i, well in wells.items():
    try:
        sim = SSDFSimulator(well_properties=well.wp, boundary_conditions=well.bc)
        x = sim.simulate()
        dp = create_data_point(well=well, sim=sim, x=x, decision_type='Perturbation')
        oil_before = dp["WOIL"].values[0]

        well.bc.w_lg = 1.5

        sim = SSDFSimulator(well_properties=well.wp, boundary_conditions=well.bc)
        x = sim.simulate()
        dp = create_data_point(well=well, sim=sim, x=x, decision_type='Perturbation')
        oil_after = dp["WOIL"].values[0]

        difference = oil_after - oil_before

        differences = differences.append({
            'well_id': i,
            'oil_before': oil_before,
            'oil_after': oil_after,
            'difference': difference
        }, ignore_index=True)

    except SimError as e:
        continue

/var/folders/ng/6xlm381s2n9g1yzbrt_z2__00000gn/T/ipykernel_78586/2475522981.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  differences = differences.append({
/var/folders/ng/6xlm381s2n9g1yzbrt_z2__00000gn/T/ipykernel_78586/2475522981.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  differences = differences.append({
/var/folders/ng/6xlm381s2n9g1yzbrt_z2__00000gn/T/ipykernel_78586/2475522981.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  differences = differences.append({
/var/folders/ng/6xlm381s2n9g1yzbrt_z2__00000gn/T/ipykernel_78586/2475522981.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  differences = differenc

In [22]:
differences

,well_id,oil_before,oil_after,difference
0,100.0,17.106061,17.408870,0.302809
1,1913.0,7.863073,8.807109,0.944036
2,1894.0,4.600271,7.304582,2.704311
3,1053.0,12.732349,14.965939,2.233590
4,1723.0,15.528862,15.989991,0.461129
5,382.0,11.350031,13.016749,1.666718
6,1466.0,7.875625,7.710757,-0.164867
7,988.0,12.092859,15.550713,3.457854
8,234.0,6.642827,10.038991,3.396165
9,1409.0,4.149496,8.266249,4.116753


### Find initial production

In [12]:
import os
print(os.path.exists("../results/"))

True


In [52]:
dataset_filename = 'data/testing/SPSA-sol_configs/sample_6.csv'
df = pd.read_csv(dataset_filename)
wells = configure_wells(df)

In [53]:
opt = SPSAOptimization(wells=wells, sigma=0., a=0, c=0)
opt.SPSA(n_sim=1)

Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive direction: [0.5 0. ]
Negative direction [0.5 0. ]
Positive state:
Total oil produced is 97.763935686672
Total water produced is 71.59331390173743, leaving a slack of 0.4066860982625684
Total gas lift used is 0.0, leaving a slack of 5.0
Negative state:
Total oil produced is 97.763935686672
Total water produced is 71.59331390173743, leaving a slack of 0.4066860982625684
Total gas lift used is 0.0, leaving a slack of 5.0


/Users/henrikaas/Library/CloudStorage/OneDrive-NTNU/Dokumenter/NTNU/Prosjektoppgave/Kode/manywells/SPSA.py:811: RuntimeWarning: invalid value encountered in divide
  gradient = (pos_L - neg_L).reshape(-1, 1) / (2 * well_directions)
CasADi - 2025-06-28 16:25:08 WARNING("S:nlp_jac_g failed: NaN detected for output jac_g_x, at nonzero index 0 (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2025-06-28 16:25:08 WARNING("S:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2025-06-28 16:25:08 WARNING("S:nlp_jac_g failed: NaN detected for output jac_g_x, at nonzero index 4092 (row 703, col 700).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2025-06-28 16:25:08 WARNING("S:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]


Could not solve for alpha
Solver status: {'iter_count': 0, 'n_call_callback_fun': 0, 'n_call_nlp_f': 0, 'n_call_nlp_g': 1, 'n_call_nlp_grad': 0, 'n_call_nlp_grad_f': 1, 'n_call_nlp_hess_l': 0, 'n_call_nlp_jac_g': 1, 'return_status': 'Invalid_Number_Detected', 'success': False, 't_proc_callback_fun': 0.0, 't_proc_nlp_f': 0.0, 't_proc_nlp_g': 2.7e-05, 't_proc_nlp_grad': 0.0, 't_proc_nlp_grad_f': 1e-06, 't_proc_nlp_hess_l': 0.0, 't_proc_nlp_jac_g': 0.000123, 't_wall_callback_fun': 0.0, 't_wall_nlp_f': 0.0, 't_wall_nlp_g': 2.725e-05, 't_wall_nlp_grad': 0.0, 't_wall_nlp_grad_f': 8.75e-07, 't_wall_nlp_hess_l': 0.0, 't_wall_nlp_jac_g': 0.000141833, 'unified_return_status': 'SOLVER_RET_UNKNOWN'}
Simulation failed for well 5 without initial guess. Trying guesses...
Simulation failed
Solver status: {'iter_count': 0, 'n_call_callback_fun': 0, 'n_call_nlp_f': 0, 'n_call_nlp_g': 1, 'n_call_nlp_grad': 0, 'n_call_nlp_grad_f': 1, 'n_call_nlp_hess_l': 0, 'n_call_nlp_jac_g': 1, 'return_status': 'Invalid

CasADi - 2025-06-28 16:25:09 WARNING("S:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]


Simulation failed
Solver status: {'iter_count': 0, 'n_call_callback_fun': 0, 'n_call_nlp_f': 0, 'n_call_nlp_g': 1, 'n_call_nlp_grad': 0, 'n_call_nlp_grad_f': 1, 'n_call_nlp_hess_l': 0, 'n_call_nlp_jac_g': 1, 'return_status': 'Invalid_Number_Detected', 'success': False, 't_proc_callback_fun': 0.0, 't_proc_nlp_f': 0.0, 't_proc_nlp_g': 0.000382, 't_proc_nlp_grad': 0.0, 't_proc_nlp_grad_f': 5e-06, 't_proc_nlp_hess_l': 0.0, 't_proc_nlp_jac_g': 0.000123, 't_wall_callback_fun': 0.0, 't_wall_nlp_f': 0.0, 't_wall_nlp_g': 0.000387084, 't_wall_nlp_grad': 0.0, 't_wall_nlp_grad_f': 3.708e-06, 't_wall_nlp_hess_l': 0.0, 't_wall_nlp_jac_g': 0.000124042, 'unified_return_status': 'SOLVER_RET_UNKNOWN'}
Simulation failed for well 5. Trying next guess...


...

CasADi - 2025-06-28 16:25:36 WARNING("S:nlp_jac_g failed: NaN detected for output jac_g_x, at nonzero index 4092 (row 703, col 700).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2025-06-28 16:25:36 WARNING("S:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]


Simulation failed
Solver status: {'iter_count': 0, 'n_call_callback_fun': 0, 'n_call_nlp_f': 0, 'n_call_nlp_g': 1, 'n_call_nlp_grad': 0, 'n_call_nlp_grad_f': 1, 'n_call_nlp_hess_l': 0, 'n_call_nlp_jac_g': 1, 'return_status': 'Invalid_Number_Detected', 'success': False, 't_proc_callback_fun': 0.0, 't_proc_nlp_f': 0.0, 't_proc_nlp_g': 6.4e-05, 't_proc_nlp_grad': 0.0, 't_proc_nlp_grad_f': 4e-06, 't_proc_nlp_hess_l': 0.0, 't_proc_nlp_jac_g': 0.000383, 't_wall_callback_fun': 0.0, 't_wall_nlp_f': 0.0, 't_wall_nlp_g': 6.3208e-05, 't_wall_nlp_grad': 0.0, 't_wall_nlp_grad_f': 3.5e-06, 't_wall_nlp_hess_l': 0.0, 't_wall_nlp_jac_g': 0.000375041, 'unified_return_status': 'SOLVER_RET_UNKNOWN'}
Simulation failed for well 5. Trying next guess...
Simulation failed
Solver status: {'iter_count': 0, 'n_call_callback_fun': 0, 'n_call_nlp_f': 0, 'n_call_nlp_g': 1, 'n_call_nlp_grad': 0, 'n_call_nlp_grad_f': 1, 'n_call_nlp_hess_l': 0, 'n_call_nlp_jac_g': 1, 'return_status': 'Invalid_Number_Detected', 'success

CasADi - 2025-06-28 16:25:36 WARNING("S:nlp_jac_g failed: NaN detected for output jac_g_x, at nonzero index 4092 (row 703, col 700).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2025-06-28 16:25:36 WARNING("S:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2025-06-28 16:25:36 WARNING("S:nlp_jac_g failed: NaN detected for output jac_g_x, at nonzero index 4092 (row 703, col 700).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2025-06-28 16:25:36 WARNING("S:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]


Simulation failed
Solver status: {'iter_count': 0, 'n_call_callback_fun': 0, 'n_call_nlp_f': 0, 'n_call_nlp_g': 1, 'n_call_nlp_grad': 0, 'n_call_nlp_grad_f': 1, 'n_call_nlp_hess_l': 0, 'n_call_nlp_jac_g': 1, 'return_status': 'Invalid_Number_Detected', 'success': False, 't_proc_callback_fun': 0.0, 't_proc_nlp_f': 0.0, 't_proc_nlp_g': 6.7e-05, 't_proc_nlp_grad': 0.0, 't_proc_nlp_grad_f': 4e-06, 't_proc_nlp_hess_l': 0.0, 't_proc_nlp_jac_g': 0.000181, 't_wall_callback_fun': 0.0, 't_wall_nlp_f': 0.0, 't_wall_nlp_g': 6.575e-05, 't_wall_nlp_grad': 0.0, 't_wall_nlp_grad_f': 3.084e-06, 't_wall_nlp_hess_l': 0.0, 't_wall_nlp_jac_g': 0.000181375, 'unified_return_status': 'SOLVER_RET_UNKNOWN'}
Simulation failed for well 5. Trying next guess...


SystemError: <built-in function nlpsol> returned a result with an exception set